# Imports

In [198]:
import gzip
import pandas as pd
from collections import Counter
import string
from collections import defaultdict
from bs4 import BeautifulSoup
import requests
import langid
import sys
from glob import glob
import os
import pickle
# For field matching
#import field_matching
from tqdm import tqdm
STOPWORDS = set(['and','not','of','the'])
import numpy as np
import random
import re, string
from glob import glob

SUMMARY_DATA_DIR ="/data/orcid/summ_out/"

# Read in Processed XML files

In [199]:
all_summ_files = [pd.read_parquet(x) for x in glob(f"{SUMMARY_DATA_DIR}/*_affiliations.parquet")]
print(len(all_summ_files))
d = pd.concat(all_summ_files)

1100


In [200]:
d.type.value_counts()

type
employment    12880514
education      8379178
invited         373988
Name: count, dtype: int64

In [201]:
# drop invited
d = d[d.type != 'invited']

In [202]:
d.shape, d.orcid.nunique()

((21259692, 9), 10095804)

In [203]:
#d.sample(25000).to_csv("sample_orcid.csv",index=False)

In [204]:
d.sample(3)

,orcid,name,type,org_name,role,country,department_name,start_year,end_year
11360,0000-0002-4764-8598,Benedict Lane,employment,TU Delft,None,NL,"Faculty of Technology, Policy & Management",2024.0,NaN
14425,0000-0002-9194-348X,Emrullah CANSU,education,Kastamonu Üniversitesi,Master's Degree,TR,Tourism Management,NaN,2018.0
13446,0009-0002-8493-4005,Lauren,employment,University of Copenhagen,None,DK,Science,NaN,NaN


In [205]:
d['affiliation'] = d.department_name
d['affiliation_type'] = 'department_name'
print(f"Not null department_name: {sum(d.affiliation.notnull())/len(d)}")

Not null department_name: 0.7367723859781223


In [206]:
print(f"""Of null department_name, percent not-null org: 
{len(d[d.department_name.isnull() & d.org_name.notnull()])/len(d[d.department_name.isnull()])}
""")

Of null department_name, percent not-null org: 
1.0



In [207]:
d.loc[d.department_name.isnull(),"affiliation"] = d.loc[d.department_name.isnull(),"org_name"] 
d.loc[d.department_name.isnull(),"affiliation_type"] = 'org_name'

## If the "department_name" matches an entry on a list of "departments" in that country (see list in paper), then don't feed the department name to the STEM classifier

In [208]:
regions_by_country = {
    "CO": [
        "Amazonas", "Antioquia", "Arauca", "Atlántico", "Bolívar", "Boyacá",
        "Caldas", "Caquetá", "Casanare", "Cauca", "Cesar", "Chocó", "Córdoba",
        "Cundinamarca", "Distrito Capital", "Guainía", "Guaviare", "Huila",
        "La Guajira", "Magdalena", "Meta", "Nariño", "Norte de Santander",
        "Putumayo", "Quindío", "Risaralda", "San Andrés y Providencia",
        "Santander", "Sucre", "Tolima", "Valle del Cauca", "Vaupés", "Vichada"
    ],
    "PE": [
        "Amazonas", "Ancash", "Apurímac", "Arequipa", "Ayacucho", "Cajamarca",
        "Callao", "Cusco", "Huancavelica", "Huánuco", "Ica", "Junín",
        "La Libertad", "Lambayeque", "Lima", "Loreto", "Madre de Dios",
        "Moquegua", "Pasco", "Piura", "Puno", "San Martín", "Tacna",
        "Tumbes", "Ucayali"
    ],
    "SV": [
        "Ahuachapán", "Cabañas", "Chalatenango", "Cuscatlán", "La Libertad",
        "La Paz", "La Unión", "Morazán", "San Miguel", "San Salvador",
        "San Vicente", "Santa Ana", "Sonsonate", "Usulután"
    ],
    "BO": [
        "Chuquisaca", "Cochabamba", "El Beni", "La Paz", "Oruro", "Pando",
        "Potosí", "Santa Cruz", "Tarija"
    ],
    "UY": [
        "Artigas", "Canelones", "Cerro Largo", "Colonia", "Durazno",
        "Flores", "Florida", "Lavalleja", "Maldonado", "Montevideo",
        "Paysandú", "Río Negro", "Rivera", "Rocha", "Salto", "San José",
        "Soriano", "Tacuarembó", "Treinta y Tres"
    ],
    "PY": [
        "Alto Paraguay", "Alto Paraná", "Amambay", "Asunción", "Boquerón",
        "Caaguazú", "Caazapá", "Canindeyú", "Central", "Concepción",
        "Cordillera", "Guairá", "Itapúa", "Misiones", "Ñeembucú",
        "Paraguarí", "Presidente Hayes", "San Pedro"
    ]
}

In [209]:
for k, v in regions_by_country.items():
    locs = [x.lower() for x in v]
    country_df = d[d.country == k]
    print(f"""Country: {k}
                N in Country: {len(country_df)}
                % department is location:
                    {len(country_df[(country_df.department_name.str.lower().isin(locs))])/len(country_df)}
            """)
    mask = (d.country == k) & (d.department_name.str.lower().isin(locs))
    d.loc[mask,"affiliation"] = d.loc[mask,"org_name"]
    d.loc[mask,"affiliation_type"] = "org_name_country_rule"

Country: CO
                N in Country: 514929
                % department is location:
                    0.21280992136780022
            
Country: PE
                N in Country: 616314
                % department is location:
                    0.35541461008511893
            
Country: SV
                N in Country: 8975
                % department is location:
                    0.16044568245125349
            
Country: BO
                N in Country: 24669
                % department is location:
                    0.17649681786857999
            
Country: UY
                N in Country: 17256
                % department is location:
                    0.07452480296708391
            
Country: PY
                N in Country: 21534
                % department is location:
                    0.06427045602303334
            


## Do a little cleaning

In [212]:
d['affiliation'] = d.affiliation.str.replace("\t"," ").str.strip()

In [213]:
# For non-English, 2 characters could reasonably be a field! But only one ... probably not

print((d.affiliation.str.len() < 2).sum())
d = d[d.affiliation.str.len()>=2]

3278


In [214]:
print(d.affiliation.isin(['n/a','none','na']).sum())
d = d[~d.affiliation.isin(['n/a','none','na'])]

533


# Set Dates and Drop Null Dates

In [215]:
print(f"Proportion no start/end year: {len(d[d.start_year.isnull() & d.end_year.isnull()])/len(d)}")

Proportion no start/end year: 0.1972161963082123


In [216]:
d = d[d.start_year.notnull() | d.end_year.notnull()]

In [217]:
d.affiliation_type.value_counts() / len(d)

affiliation_type
department_name          0.795448
org_name                 0.187196
org_name_country_rule    0.017356
Name: count, dtype: float64

In [218]:
end_no_start = (d.start_year.isnull() &  ~d.end_year.isnull())
print(f"%age with end year but no start year: {end_no_start.sum()/len(d)}")

%age with end year but no start year: 0.02411814149855862


In [219]:
d.loc[end_no_start,"start_year"] = d.loc[end_no_start].end_year
d.loc[end_no_start,"end_year"] = np.nan

In [220]:
d.shape, d.orcid.nunique()

((17063877, 11), 7110339)

In [221]:
d.end_year.isnull().sum(), len(d), d.end_year.isnull().sum()/ len(d)

(7256909, 17063877, 0.42527902656588534)

# Translation

In [64]:
tr = dict((ord(x), None) for x in string.punctuation if x != " " and x != "-")

def lookup(myjson, k):
  # return myjson[k]
  if '.' in k:
    # jpath path
    ks = k.split('.')
    v = myjson
    for k in ks: 
        if not v:
            return ""
        v = v.get(k,{})
    return v or ""
  return myjson.get(k,"")


def get_terms(text, term_set =None):
    spl = re.split("[ /+,]",text.lower())
    t = set([x.translate(tr) for x in spl])
    if term_set:
        return t & term_set
    return t

In [65]:
#NSF Fields
import requests 
from collections import defaultdict,Counter
from bs4 import BeautifulSoup
tab = requests.get("https://wayback.archive-it.org/5902/20231215174930/https://www.nsf.gov/statistics/2017/nsf17306/datatables/tab-13.htm")
s = BeautifulSoup(tab.text)
tab = s.find("table",{"id" : "data-table"})

def get_text_indent(row):
    text = row.text.replace(", general","").replace(", other","").lower().strip()
    indent = int(row['class'][0].replace("indent_",""))
    return text, indent

nsf_field_hierarchy = defaultdict(list)
nsf_terms = set()
rows = tab.find_all("th", {"scope" : "row"})

# stupid but it works
i = 1
while i < len(rows):
    # get my info
    curr_text, curr_indent  = get_text_indent(rows[i])
    # make sure this isn't already in the tree
    if curr_text not in nsf_terms:
        nsf_terms.add(curr_text)
        for j in reversed(range(i)):
            prev_text, prev_indent = get_text_indent(rows[j])
            if curr_indent - prev_indent == 1:
                nsf_field_hierarchy[prev_text].append(curr_text)
                break
    #else:
        #print rows[i].text, curr_text, ' already in tree'
    i += 1

nsf_termlist = Counter()
for x in nsf_terms:
    nsf_termlist.update([x for x in get_terms(x) if x not in STOPWORDS])
nsf_termlist = set(nsf_termlist.keys())

In [66]:
# Wikipedia Fields
#https://en.wikipedia.org/wiki/Outline_of_academic_disciplines
def get_text_indent_wiki(row):
    indent = -1
    if row.startswith("==="):
        indent = 1
    elif row.startswith("=="):
        indent = 0
    elif row.startswith("'''"):
        indent = 2
    elif row.startswith("***"):
        indent = 5
    elif row.startswith("**"):
        indent = 4
    elif row.startswith("*"):
        indent = 3
    else:
        pass
        #print "NOPE", row
    text = row.replace("*","").replace("=","").lower().strip()
    
    text = text.replace("'''","")

    text = re.sub("(#|\|)[a-z A-Z\(\)]+\]\]","",text)
    text = text.replace("[[","").replace("]]","").strip()
    if text.rfind("(outline") != -1:
        text = text[:text.rfind("(outline")]
    return text.strip(), indent

wiki_field_hierarchy = defaultdict(list)
rows = [get_text_indent_wiki(x) for x in open("data/wiki_page.txt").readlines()]
rows = [x for x in rows if x[1] != -1]
wiki_terms = set()

# stupid but it works
i = 1
while i < len(rows):
    # get my info
    curr_text, curr_indent  = rows[i]
    # make sure this isn't already in the tree
    if curr_text not in wiki_terms:
        wiki_terms.add(curr_text)
        for j in reversed(range(i)):
            prev_text, prev_indent = rows[j]
            if curr_indent - prev_indent > 0:
                wiki_field_hierarchy[prev_text].append(curr_text)
                break
    i += 1
    
wiki_termlist = Counter()
for x in wiki_terms:
    wiki_termlist.update([x for x in get_terms(x) if x not in STOPWORDS])
wiki_termlist = set(wiki_termlist.keys())


for term in wiki_terms:
    if term not in wiki_field_hierarchy:
        wiki_field_hierarchy[term] = []
        
        
# Archaeology is a social science discipline - https://en.wikipedia.org/wiki/Archaeology
# So is communications
wiki_field_hierarchy['social sciences'].append("archaelogy")
wiki_terms.add("archaeology")
wiki_field_hierarchy['social sciences'].append("communications")
wiki_terms.add("communications")

# if Music history is a subset of Music, art history should be a subset of art
wiki_field_hierarchy["visual arts"].append("art history")
wiki_terms.add("art history")

# Classics == classical literature
wiki_field_hierarchy["history of literature"].append("classics")
wiki_field_hierarchy["history of literature"].append("classical literature")
wiki_terms.add("classics")
wiki_terms.add("classical literature")

# If English lit is under language and arts, so is spanish
wiki_field_hierarchy["languages and literature"].append("spanish literature")
wiki_terms.add("spanish literature") 

In [67]:
remove_list = {'a',''
  'as',
 'early',
 'or',
 'tv',
 'us',
'than',
'chemistrya',
'non-se',
'field',
'list','2017'}
nsf_termlist = nsf_termlist - remove_list
wiki_termlist = wiki_termlist - remove_list

In [68]:
# Bring back in cleaned version and count number of terms in each individual's department
manual_termlist = {x.split("\t")[0] for x in open("manual_keywords.tsv")}

In [69]:
# Write out the fields that we need to translate to English
non_eng_affl = d[~d.country.isin(["US","UK", "GB"])].affiliation.unique()
print(len(non_eng_affl))


3091209


In [70]:
nsf_termlist = {x for x in nsf_termlist if len(x) > 3}
wiki_termlist = {x for x in wiki_termlist if len(x) > 3}

In [71]:
# with open("all_terms.txt","w") as af:
#     for term in nsf_termlist | wiki_termlist | manual_termlist:
#         af.write(term + "\n")

In [ ]:
to_translate = set()
for i,x in enumerate(non_eng_affl):
    if i % 50000 == 0:
        print(i)
    if x and type(x) is str: 
        terms = get_terms(x)
        if (len(terms & nsf_termlist) or 
            len(terms & wiki_termlist) or 
            len(terms & manual_termlist) or 
            langid.classify(x)[0] == 'en'):
                continue
        to_translate.add(x.lower())

## Do google translate

In [117]:
already_translated = (pd.read_parquet("data/final_final_translation_2025.parquet").from_field.values.tolist()+
pd.read_parquet("data/translated_fieldnames_2025_v2.parquet").input.values.tolist()+
                      pd.read_parquet("data/translated_fieldnames_2025_v3.parquet").input.values.tolist())
                                
print(len(set(to_translate)), len(set(already_translated)), 
      len(set(to_translate) - set(already_translated)))


18019 1343270 0


In [100]:
to_translate = list(set(to_translate) - set(already_translated))

In [101]:
from google.oauth2 import service_account
from google.cloud import translate_v2 as translate

credentials = service_account.Credentials.from_service_account_file(
    'YOUR_GOOGLE_PROJECT_CREDS'
)
translate_client = translate.Client(credentials=credentials)

In [102]:
sum(map(len, to_translate))

665959

In [103]:
import math 
import time
# Translate the list
batch_size = 100
translations = []
num_batches = math.ceil(len(to_translate) / batch_size)

for i in tqdm(range(0, len(to_translate), batch_size), 
              total=num_batches,
              desc="Translating batches"):
    batch = to_translate[i:i + batch_size]
    results = translate_client.translate(batch, target_language='en')
    translations.extend(results)
    time.sleep(5)

Translating batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 181/181 [16:14<00:00,  5.39s/it]


In [105]:
len(translations)

18019

In [106]:
pd.DataFrame(translations).to_parquet("data/translated_fieldnames_2025_v4.parquet")

In [107]:
pd.DataFrame(translations).sample(10)

,translatedText,detectedSourceLanguage,input
9847,Ribatejo Technical and Vocational School,pt,escola técnica e profissional do ribatejo
3845,inter-institutional skills assessment center,fr,centre interinstitutionnel de bilan de compéte...
3044,Grodno University of Applied Sciences,it,grodno l-m university
7196,coordination of private universities in region II,id,koordinasi perguruan tinggi swasta wilayah ii
7602,Melgaço Municipality,pt,melgaço municipality
7026,Icons Institute for Research in Communication ...,es,iconos instituto de investigacion en comunicac...
1119,Lucas de Oliveira &amp; Singer Associated Lawyers,pt-PT,lucas de oliveira & singer advogados associados
3244,"Institute of Archatology: Odessa, Kyiv, Ukraine",ru,"institute of archatology : odessa, kyiv, украина"
11217,"mrhru-badouni, nirth-jabalpur",en,"mrhru-badouni, nirth-jabalpur"
1677,corporation goes,es,corporación goes


## Combine, create final translated dataset

In [113]:
just_translated = pd.concat([pd.read_parquet("data/translated_fieldnames_2025_v2.parquet"),
                              pd.read_parquet("data/translated_fieldnames_2025_v3.parquet"),
                              pd.read_parquet("data/translated_fieldnames_2025_v4.parquet")])
just_translated.columns = ['to_field','source_lang_translate','from_field']

In [114]:
original = pd.read_parquet("data/final_translation_2025.parquet")

In [116]:
pd.concat([just_translated,original]).to_parquet("data/final_final_translation_2025.parquet")

# Gender inference

## Clean names

In [118]:
# TAKEN FROM https://github.com/cthoyt/orcid_downloader/blob/main/src/orcid_downloader/name_utils.py!!!!

punc_trans = str.maketrans('', '', string.punctuation)
def clean_name(name: str) -> str:
    """Clean a name string.

    :param name: A full name
    :return: A cleaned full name, e.g., stripped of titles and suffixes
    """
    # strip titles like Dr. and DR. from beginning of all names/aliases
    # strip post-titles Francess Dufie Azumah (DR.)
    lower = name.lower()
    if lower.startswith("professor "):
        name = name[len("professor ") :].strip()
    if lower.startswith("prof."):
        name = name[len("prof.") :].strip()
    if lower.startswith("dr.-ing."):
        name = name[len("dr.-ing.") :]
    if lower.startswith("dr "):
        name = name[len("dr ") :]
    if lower.startswith("dr."):
        name = name[len("dr.") :].strip()

    for suffix in [
        "(dr)",
        "(dr.)",
        ", m.d.",
        ", phd",
        ", md",
        ", mph",
        ", ph.d.",
        ", ms",
    ]:
        if lower.endswith(suffix):
            name = name.removesuffix(suffix).strip()

    name = name.replace('"', "")
    name = name.strip("/")
    name = name.strip("\\")

    # Fix lazily written in all caps or lower
    if name == name.lower() or name == name.upper():
        name = name.title()

    return name


In [119]:
d['clean_name'] = d.name.apply(clean_name)
d['clean_name'] = d.clean_name.str.replace(
    r"^[^\w\s]+|[^\w\s]+$",
    "",
    regex=True
)
d['clean_name'] = d.clean_name.str.replace("^(([a-z][a-z]?[.]?)|(prof[.]?)) ", "", 
                               regex=True, 
                               flags=re.IGNORECASE)
d['clean_name'] = d.clean_name.str.replace("^(([a-z][a-z]?[.]?)|(prof[.]?)) ", "", 
                               regex=True, 
                               flags=re.IGNORECASE)

In [120]:
# Find rows with only spaces/punctuation
mask = d['clean_name'].str.match(r'^[\s\W]*$', na=False)

# Filter the dataframe
print(f"Names with only spaces or punctuation: {mask.sum()}")
d = d[~mask]

Names with only spaces or punctuation: 912


In [121]:
# only one alpha character
mask = d.clean_name.str.match(r'^[a-zA-Z]$', na=False)
print(f"Names with only one alpha character: {mask.sum()}")
d = d[~mask]

Names with only one alpha character: 735


In [122]:
all_names = d.clean_name.unique().tolist()

In [123]:
len(all_names)

6230821

## Run NomQuam

In [124]:
import nomquamgender as nqg
model = nqg.NBGC()
n = model.annotate(all_names, as_df=True)

In [126]:
# Explore missing and write out for other tool
missing = n.loc[n['p(gf)'].isna()]
missing.to_csv("missing_names.csv",index=False)
missing['used'].value_counts()[:10]

used
d.      444
m.      418
s.      368
m       332
dra.    287
a.      278
mst.    228
c.      224
moh.    191
dr.)    187
Name: count, dtype: int64

In [127]:
# How many are missing
n.loc[n['p(gf)'].isna()].shape

(541217, 5)

In [128]:
n.loc[n['p(gf)'].isna()].sample(10)

,given,used,sources,counts,p(gf)
2194459,Janmenjoy Nayak,janmenjoy,0,0,NaN
1281572,Khumblani Mnqiwu,khumblani,0,0,NaN
4612308,Hemachandran Veerasamy,hemachandran,0,0,NaN
2710754,Hassinger Adam Mercado Puente,hassinger,0,0,NaN
1782386,Shenrui Zhang,shenrui,0,0,NaN
4269251,Quancan Huang,quancan,0,0,NaN
2139667,Voyna Andriy,voyna,0,0,NaN
965892,Yeci Lulu Tafur Saenz,yeci,0,0,NaN
903453,Feng-Fan Yang,feng-fan,0,0,NaN
830790,hamreen Ahmad Ahmad,hamreen,0,0,NaN


In [129]:
# Look at per-country missing names
m = pd.merge(n,d[['clean_name','country']], 
             left_on="given",
             right_on="clean_name",how="left")
m['has_val'] = m['p(gf)'].isna()

gb = m.groupby(['country','has_val']).size().reset_index()
gb.columns = ['country','is_null','cnt']
gb.to_csv("country_to_missing_names.csv",index=False)

## Run Namsor on missing names

In [130]:
missing_names = n.loc[n['p(gf)'].isna()].given.values
already_done = pd.read_parquet("data/namsor_gender_outputs.parquet")
still_missing = list(set(missing_names)- set(already_done.name))

In [131]:
len(still_missing)

12

In [132]:
import requests
from tqdm import tqdm

API_KEY = "YOUR_API_KEY"
BASE_URL = "https://v2.namsor.com/NamSorAPIv2/api2/json"

headers = {
    "X-API-KEY": API_KEY,
    "Accept": "application/json",
    "Content-Type": "application/json"
}


def get_gender_full_batch_safe(full_names_list, batch_size=100):
    """Process full names in batches with error handling"""
    all_results = []
    
    for i in tqdm(range(0, len(full_names_list), batch_size)):
        batch = full_names_list[i:i+batch_size]
        names = [{"name": name} for name in batch]
        
        url = f"{BASE_URL}/genderFullBatch"
        payload = {"personalNames": names}
        
        try:
            response = requests.post(url, json=payload, headers=headers)
            response.raise_for_status()
            results = response.json()
            all_results.extend(results['personalNames'])
        except requests.exceptions.RequestException as e:
            print(f"Error processing batch {i}: {e}")
            # Add None values for failed batch
            all_results.extend([None] * len(batch))
    
    return all_results

# Usage
results = get_gender_full_batch_safe(still_missing)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.42it/s]


In [133]:
pd.concat([pd.DataFrame(results),already_done]).to_parquet("data/namsor_gender_outputs.parquet")

In [134]:
namsor_results = pd.concat([pd.DataFrame(results),already_done])
namsor_results.head()

,script,id,name,likelyGender,genderScale,score,probabilityCalibrated
0,LATIN,bcdceb54-3772-4b98-91be-8c780e928e60,Valdiele Silva,female,0.397811,4.795512,0.698905
1,LATIN,6a68646d-2b69-497f-bc04-1c2071d8fd1c,Dachaphon Kealkaew,male,-0.346527,2.796292,0.673264
2,LATIN,17764c43-1f4d-40fc-94da-2718105403ab,NGOUAT Dominique,male,-0.256714,1.942457,0.628357
3,LATIN,b6733866-2411-430d-a975-6fd9a11b6bb9,Svitenok Mariia,female,0.069563,0.189652,0.534782
4,LATIN,247bdbfd-fdd0-43bd-9361-e1f1e1b9ce28,4K vip Đá gà,male,-0.781462,7.890642,0.890731


## Construct final names dataset

In [135]:
# Match Namsor probabilities to NomQuam
def gen_miss(x):
    if x['likelyGender'] == 'female':
        return x['probabilityCalibrated']
    return 1-x['probabilityCalibrated']
namsor_results['p(gf)'] = namsor_results.apply(gen_miss,axis=1)

In [136]:
namsor_results['given'] = namsor_results.name

In [137]:
final_names = pd.concat([n[['given','p(gf)']][~n['p(gf)'].isna()],
           namsor_results[['given','p(gf)']]],
          axis=0)

In [139]:
final_names.shape, len(all_names), final_names[~final_names.given.duplicated()].shape

((6318821, 2), 6230821, (6318821, 2))

In [140]:
d.shape, d.merge(final_names, left_on="clean_name", right_on="given", how ='left').shape

((17062230, 12), (17062230, 14))

In [141]:
d = d.merge(final_names, left_on="clean_name", right_on="given", how ='left')

In [142]:
d[d['p(gf)'].isnull()].shape

(0, 14)

# Complete translation

In [143]:
translated = pd.read_parquet("data/final_final_translation_2025.parquet")
translated.from_field = translated.from_field.str.lower()
translated.to_field = translated.to_field.str.lower()
translated.drop_duplicates("from_field",inplace=True)

In [144]:
d['affiliation'] = d['affiliation'].str.lower()


In [145]:
d = pd.merge(d, translated,left_on="affiliation",right_on="from_field",how="left")
d['translated_affiliation'] = d.apply(lambda x : x['affiliation'] if pd.isnull(x['to_field']) else x['to_field'],axis=1)
d.shape

(17062230, 18)

In [156]:
pd.set_option('display.max_colwidth', None) # Use None or -1 (for pandas < 1.0)


In [158]:
def clean_fieldnames(name):
    if not name:
        return None
    name = name.lower()
    name = name.replace("department of","")
    name = name.replace("dept of", "")
    name = name.replace("faculty of", "")
    name = name.replace("school of", "")
    name = name.replace("college of", "")
    name = name.replace("institute for", "")
    name = name.replace("institute of", "")
    name = name.replace("the ", "")
    name = name.replace("  "," ")
    name = name.replace("\t", " ").replace("\n"," ")
    name = name.replace("(","").replace(")","")
    return(name.strip())

d = d.assign(clean_affiliation = d.translated_affiliation.apply(clean_fieldnames))

# Drop people with excessive role counts

In [159]:
# How many affiliations per person
affiliations_per_person = d.groupby("orcid").size()

In [160]:
affiliations_per_person.shape

(7071222,)

In [161]:
affl_percentage_cum = (affiliations_per_person.value_counts()/float(len(affiliations_per_person))).cumsum()

In [162]:
# Cumsum of affiliations per person - we can toss ppl with > 12 affiliations and lose only .05% of the data
affl_percentage_cum.head(15)

#what are these 2 affiliations?

1     0.506190
2     0.694723
3     0.801193
4     0.874170
5     0.919692
6     0.949369
7     0.968634
8     0.981175
9     0.989229
10    0.994379
11    0.997734
12    1.000000
Name: count, dtype: float64

In [163]:
# Remove ppl with > 12 affiliations

d = d[d.orcid.isin(affiliations_per_person[affiliations_per_person < 13].index.values)]
print(d.shape)
# How many ppl. do we have a this point?
d.orcid.nunique()

(16430063, 19)


7071222

In [164]:
d.country.nunique()

250

In [165]:
d.shape

(16430063, 19)

# Write out

In [172]:
d.sample(10)

,orcid,name,type,org_name,role,country,department_name,start_year,end_year,affiliation,affiliation_type,clean_name,given,p(gf),to_field,source_lang_translate,from_field,translated_affiliation,clean_affiliation
15663109,0000-0002-3181-1442,Íris Ascenção,education,Charles University,None,CZ,None,2018.0,2019.0,charles university,org_name,Íris Ascenção,Íris Ascenção,0.988,charles university,en,charles university,charles university,charles university
3120770,0009-0000-4429-8011,Esther Mashabile,employment,University of the Free State,Post-Doctoral Research Fellow,ZA,Centre of Development Support: Faculty of Economic and Management Sciences,2025.0,2026.0,centre of development support: faculty of economic and management sciences,department_name,Esther Mashabile,Esther Mashabile,0.984,NaN,NaN,NaN,centre of development support: faculty of economic and management sciences,centre of development support: economic and management sciences
11495444,0000-0002-3931-9874,María Falconí Abad,employment,Universidad de Cuenca,Docente/investigadora,EC,Carrera de Sociología,2015.0,NaN,carrera de sociología,department_name,María Falconí Abad,María Falconí Abad,0.953,sociology career,None,carrera de sociología,sociology career,sociology career
13078515,0009-0000-6185-2336,Mavuto Kambochola Banda,employment,University of Hull,Part-time Teaching Assistant,GB,"School of Environment Sciences, Wilberforce Institute",2022.0,2022.0,"school of environment sciences, wilberforce institute",department_name,Mavuto Kambochola Banda,Mavuto Kambochola Banda,0.500,NaN,NaN,NaN,"school of environment sciences, wilberforce institute","environment sciences, wilberforce institute"
13386243,0000-0003-1830-7900,Sally Brady,employment,HCA Laboratories Ltd,Consultant Biochemist / Clinical Lead,GB,Clinical Biochemistry,2022.0,NaN,clinical biochemistry,department_name,Sally Brady,Sally Brady,0.984,NaN,NaN,NaN,clinical biochemistry,clinical biochemistry
12947962,0000-0002-8066-4549,Carlos Santos,education,Ashridge Business School,Strategic Leadership and Change Management,GB,None,2019.0,2022.0,ashridge business school,org_name,Carlos Santos,Carlos Santos,0.000,NaN,NaN,NaN,ashridge business school,ashridge business school
10499930,0000-0003-1848-2506,Valentin Muresan,employment,Movidius,Design Centre Manager,RO,Software Division,2005.0,2016.0,software division,department_name,Valentin Muresan,Valentin Muresan,0.002,NaN,NaN,NaN,software division,software division
5607201,0000-0002-1037-7528,W. Blake Hawley,employment,University of Alabama in Huntsville,Teaching Assistant,US,Chemical and Materials Engineering,2014.0,2014.0,chemical and materials engineering,department_name,Blake Hawley,Blake Hawley,0.062,NaN,NaN,NaN,chemical and materials engineering,chemical and materials engineering
9150671,0009-0001-5127-6811,Churl Hong Chun,employment,Wonkwang University Medical Center,Professor,KR,Orthopaedics,1995.0,NaN,orthopaedics,department_name,Churl Hong Chun,Churl Hong Chun,0.000,NaN,NaN,NaN,orthopaedics,orthopaedics
13498556,0000-0001-8730-2375,Faton M. Berisha,education,University of Prishtina,PhD,AL,Department of Mathematics,1998.0,NaN,department of mathematics,department_name,Faton M. Berisha,Faton M. Berisha,0.000,NaN,NaN,NaN,department of mathematics,mathematics


In [173]:
d.to_parquet("data/final_cleaning_dataset.parquet")